In [1]:
import os,csv,requests,lxml
from bs4 import BeautifulSoup

## Scrap all books
https://books.toscrape.com/

In [2]:
BooksList = []
def main():
    for i in range(1,51):
        bookWebsiteURL = f'https://books.toscrape.com/catalogue/page-{i}.html'
        response = requests.get(bookWebsiteURL)
        soup = BeautifulSoup(response.content,'lxml')
        onePage(soup.find('section').find('ol',{'class':'row'}))
        
def onePage(page):
    Books = page.find_all('li',{'class':'col-xs-6 col-sm-4 col-md-3 col-lg-3'})
    for book in Books:
        onebook(book)
    storeInCSV(BooksList)
    BooksList.clear()

def onebook(book):
    bookLink = 'https://books.toscrape.com/catalogue/'+book.find('div',{'class':'image_container'}).find('a').attrs['href']
    bookRate = book.find('p').attrs['class'][-1]
    bookName = book.find('h3').find('a').attrs['title']
    bookPrice = book.find('div',{'class':'product_price'}).find('p').text[1:]
    bookStatus = book.find('div',{'class':'product_price'}).find('p',{'class':'instock availability'}).text.strip()
    BooksList.append((bookName,bookRate,bookStatus,bookPrice,bookLink))

def storeInCSV(books):
    m = 'a' if os.path.exists('AllBooks.csv') else 'w'
    with open('AllBooks.csv',mode=m,newline='',encoding='utf-8') as f:
        w = csv.writer(f)
        if m == 'w':
            w.writerow(['Book Name','Book Rate','Book Status','Book Price','Book Link'])
        w.writerows(books)
main()

# *********************************************************************************************

# Scrap all books for each category
https://books.toscrape.com/

In [3]:
BooksList = []
AllCategories= dict()
def main():
    bookWebsiteURL = f'https://books.toscrape.com/catalogue/page-1.html'
    response = requests.get(bookWebsiteURL)
    soup = BeautifulSoup(response.content,'lxml')
    allCategories(soup.find('div',{'class':'container-fluid page'}).find('div',{'class':'side_categories'}).find('ul').find('ul').find_all('li'))
    for k,v in AllCategories.items():
        oneCategory(k,v)

def oneCategory(title,link):
    numOfPages = 1
    response = requests.get(link)
    soup = BeautifulSoup(response.content,'lxml')
    try :
        numOfPages = int(soup.find('div',{'class':'container-fluid page'}).find('div',{'class':'row'}).find('ul',{'class','pager'}).find('li',{'class','current'}).text.strip().split(' ')[-1])
    except:
        pass
    for pgn in range(1,numOfPages+1):
        if pgn !=1:
            newLink = link.replace('index' , f'page-{pgn}')
            response = requests.get(newLink)
            soup = BeautifulSoup(response.content,'lxml')
        onePage(title,soup.find('section').find('ol',{'class':'row'}))

def onePage(cat,page):        
    Books = page.find_all('li',{'class':'col-xs-6 col-sm-4 col-md-3 col-lg-3'})
    for book in Books:
        onebook(cat,book)
    storeInCSV(BooksList)
    BooksList.clear()

def onebook(cat,book):
    bookLink = 'https://books.toscrape.com/catalogue/'+book.find('div',{'class':'image_container'}).find('a').attrs['href']
    bookRate = book.find('p').attrs['class'][-1]
    bookName = book.find('h3').find('a').attrs['title']
    bookPrice = book.find('div',{'class':'product_price'}).find('p').text[1:]
    bookStatus = book.find('div',{'class':'product_price'}).find('p',{'class':'instock availability'}).text.strip()
    BooksList.append((cat,bookName,bookRate,bookStatus,bookPrice,bookLink))

def storeInCSV(books):
    m = 'a' if os.path.exists('AllBooks.csv') else 'w'
    with open('AllBooksWithCategories.csv',mode=m,newline='',encoding='utf-8') as f:
        w = csv.writer(f)
        if m == 'w':
            w.writerow(['Category Name','Book Name','Book Rate','Book Status','Book Price','Book Link'])
        w.writerows(books)

def allCategories(categories):
    global AllCategories
    AllCategories = {category.find('a').text.strip() : 'https://books.toscrape.com/catalogue/'+category.find('a').attrs['href'] for category in categories}
#     for category in categories:
#         catName = category.find('a').text.strip()
#         catLink = 'https://books.toscrape.com/catalogue/'+category.find('a').attrs['href']
#         AllCatergories[catName] = catLink

main()